In [ ]:
import torch
import numpy as np
from torch.nn import functional as F
from transformers import XLNetModel
import math

### NN model

In [ ]:
class NN(torch.nn.Module):
    def __init__(self):
        super(NN, self).__init__()
        self.xlnet = XLNetModel.from_pretrained('XLNet-path')
        for param in self.xlnet.parameters():
            param.requires_grad = False
        self.linear_1 = torch.nn.Linear(768,128)
        self.linear_2 = torch.nn.Linear(20,128)
        self.linear_3 = torch.nn.Linear(256,3)
        self.dropout = torch.nn.Dropout(0.4)
        self.act = torch.nn.Tanh()

    def forward(self,x1,mask,x2):
        encoder_out = self.xlnet(x1,attention_mask=mask)[0]
        encoder_out,_ = torch.max(encoder_out,dim=1)
        out_1 = self.linear_1(encoder_out)
        out_1 = self.dropout(out_1)
        
        out_2 = self.linear_2(x2)
        out_2 = self.dropout(out_2)
        x = torch.concat([out_1,out_2],dim=1)
        out = self.linear_3(x)
        out = F.sigmoid(out)
        return out

### BiLSTM-CNN model

In [ ]:
class BiLSTM_CNN(torch.nn.Module):
    def __init__(self):
        super(BiLSTM_CNN, self).__init__()
        self.xlnet = XLNetModel.from_pretrained('XLNet-path')
        for param in self.xlnet.parameters():
            param.requires_grad = False
        self.bilstm = torch.nn.LSTM(
            768,
            128,
            num_layers=2,
            batch_first=True,
            dropout=0.4,
            bidirectional=True
        )
        self.cnn = torch.nn.Sequential(
            torch.nn.Conv1d(in_channels=20, out_channels=128, kernel_size=6,padding='same'),
            torch.nn.ReLU(),
            torch.nn.BatchNorm1d(128),
            torch.nn.Conv1d(128,128,kernel_size=6,padding='same'),
            torch.nn.ReLU(),
        )
        self.dropout = torch.nn.Dropout(0.5)
        self.linear_1 = torch.nn.Linear(256,128)
        self.linear_2 = torch.nn.Linear(256,3)
        self.flatten = torch.nn.Flatten()
        self.act = torch.nn.Tanh()

    def forward(self,x1,mask,x2):
        encoder_out = self.xlnet(x1,attention_mask=mask)[0]
        encoder_out,_ = torch.max(encoder_out,dim=1)
        encoder_out = encoder_out.reshape(encoder_out.shape[0],1,-1)
        out_1,_ = self.bilstm(encoder_out)
        out_1 = self.dropout(out_1)
        out_1 = out_1[:, -1, :]
        out_1 = self.linear_1(out_1)
        
        x2 = x2.view(x2.shape[0],-1,1)
        out_2 = self.cnn(x2)
        out_2 = self.dropout(out_2)
        out_2 = self.flatten(out_2)
        x = torch.cat([out_1,out_2],dim=1)
        out = self.linear_2(x)
        out = F.sigmoid(out)
        return out

### DeleSmell model

In [ ]:
class GRU_Attention(nn.Module):
    def __init__(self):
        super(GRU_Attention, self).__init__()
        self.grus = nn.Sequential(
            nn.GRU(input_size=768,hidden_size=128,num_layers=2,batch_first=True,dropout=0.5,bidirectional=True),
        )
        self.linear = nn.Linear(256,128)
        self.flatten = nn.Flatten()
        self.dropout = nn.Dropout(0.5)
        self.q = nn.Linear(256,256)
        self.k = nn.Linear(256,256)
        self.v = nn.Linear(256,256)

    def forward(self,x):
        x = x.view(x.shape[0],1,-1)
        out,_ = self.grus(x)
        out = self.dropout(out)
        query = self.q(out)
        key = self.k(out)
        value = self.v(out)
        attention = F.softmax(query @ key.transpose(1, 2) / math.sqrt(query.size(2)), dim=-1) @ value
        out = self.flatten(attention)
        out = self.linear(out)
        return out
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv1d(20,128,kernel_size=6,padding='same'),
            nn.ReLU(),
        )
        self.dropout = nn.Dropout(0.5)
        self.flatten = nn.Flatten()

    def forward(self,inputs):
        inputs = inputs.view(inputs.shape[0],-1,1)
        out = self.conv(inputs)
        out = self.dropout(out)
        out = self.flatten(out)
        return out

class deleSmell(nn.Module):
    def __init__(self):
        super(deleSmell, self).__init__()
        self.xlnet = XLNetModel.from_pretrained('XLNet-path')
        for param in self.xlnet.parameters():
            param.requires_grad = False
        self.cnn = CNN()
        self.gru_attention = GRU_Attention()
        self.linear = nn.Linear(256,3)
        self.flatten = nn.Flatten()

    def forward(self,x1,mask,x2):
        encoder_out = self.xlnet(x1,attention_mask=mask)[0]
        encoder_out,_ = torch.max(encoder_out,dim=1)
        out_1 = self.gru_attention(encoder_out)
        out_2 = self.cnn(x2)
        x = torch.cat([out_1,out_2],dim=1)
        out = self.linear(x)
        out = F.sigmoid(out)
        return out